# Selección y filtrado de datos

<img src="imagenes/proceso.png" style="width:60em; margin: 0 auto;"/>

Una vez que se han cargado los datos de una fuente, se inicia el proceso de la selección de datos. Normalmente, un conjunto de datos se recoge con más de un fin, por lo que no todas las variables recogidas o las entradas individuales son de interés para un análisis concreto. Adicionalmente, un determinado análisis puede necesitar datos que cumplan una serie de características, por lo que hay que centrarse en ellos. 

---

# Índice
[Selección por nombre de columna y valores del índice](#Selección-por-nombre-de-columna-y-valores-del-índice) <br/>
[Selección por filtrado de columna](#Selección-por-filtrado-de-columna) <br/>
[Conclusiones](#Conclusiones) <br/>

---

Supongamos, por ejemplo, que tenemos la siguiente tabla de datos de suministro eléctrico a un hogar:
<table>
    <tr><th>Hora</th><th>Consumo máximo (kW)</th><th>Consumo acumulado (kWh)</th><th>Precio (€/kWh)</th><th>Tarifa</th></tr>
    <tr><td>14:00</td><td>3.2</td><td>1.5</td><td>0.179</td><td>LLANO</td></tr>
    <tr><td>15:00</td><td>2.5</td><td>0.9</td><td>0.172</td><td>LLANO</td></tr>
    <tr><td>16:00</td><td>2.1</td><td>0.1</td><td>0.181</td><td>LLANO</td></tr>
    <tr><td>17:00</td><td>2.6</td><td>0.1</td><td>0.202</td><td>LLANO</td></tr>
    <tr><td>18:00</td><td>5.6</td><td>1.2</td><td>0.302</td><td>PUNTA</td></tr>
    <tr><td>19:00</td><td>1.4</td><td>0.2</td><td>0.314</td><td>PUNTA</td></tr>
</table>
Si queremos monitorizar los picos de consumo, sólo necesitaremos la columna <code>Consumo máximo (kW)</code>, pudiendo descartar el resto para ahorrar memoria. Por otro lado, si queremos calcular la tarifa total acumulada, sólo tendremos que usar las columnas <code>Consumo acumulado (kWh)</code> y <code>Precio (€/kWh)</code>. Y si sólo la queremos calcular en horario llano, necesitaremos tomar sólo las primeras cuatro filas. En esta libreta veremos todas las operaciones necesarias para filtrar los datos que nos interesen. Y para ello, vamos a volver sobre la librería Pandas y la potente herramienta <code>.loc[]</code>.

In [2]:
import pandas as pd
from datetime import datetime 
import matplotlib.pyplot as plt

## Selección por nombre de columna y valores del índice

En primer lugar, leemos un fichero CSV con los datos de consumo.

In [3]:
consumo = pd.read_csv("datos/consumo.csv", index_col=0, parse_dates=True)

Nótese que se ha incluido un nuevo parámetro <code>parse_dates</code>. Esto es necesario para indicarle a Pandas que busque datos de marca temporal y los traduzca a objetos temporales. De otro modo, Pandas los puede interpretar como cadenas de texto.

Si sabemos de antemano que sólo nos interesan algunas columnas o filas determinadas, la selección es muy fácil. Ya hemos visto algunos ejemplos al introducir Pandas. Para seleccionar columnas, no tenemos más que pasar una lista de los índices que nos interesan:

In [4]:
columnas = ['Consumo acumulado (kWh)', 'Precio (€/kWh)']
datos_interes = consumo.loc[:,columnas]
datos_interes

,Consumo acumulado (kWh),Precio (€/kWh)
2021-09-08 14:00:00,1.5,0.179
2021-09-08 15:00:00,0.9,0.172
2021-09-08 16:00:00,0.1,0.181
2021-09-08 17:00:00,0.1,0.202
2021-09-08 18:00:00,1.2,0.302
2021-09-08 19:00:00,0.2,0.314


Alternativamente, podemos usar <code>iloc</code> para seleccionar las columnas por su orden (empezando por 0):

In [5]:
consumo.iloc[:,[1,2]]

,Consumo acumulado (kWh),Precio (€/kWh)
2021-09-08 14:00:00,1.5,0.179
2021-09-08 15:00:00,0.9,0.172
2021-09-08 16:00:00,0.1,0.181
2021-09-08 17:00:00,0.1,0.202
2021-09-08 18:00:00,1.2,0.302
2021-09-08 19:00:00,0.2,0.314


Ambas formas son válidas, y cada una será de mayor utilidad en cada escenario particular. Para seleccionar las filas, podemos proceder de una forma similar, teniendo en cuenta el formato del índice, que en este caso es temporal:

In [6]:
desde = datetime.strptime("2021-09-08 16:00:00", "%Y-%m-%d %H:%M:%S")
hasta = datetime.strptime("2021-09-08 18:00:00", "%Y-%m-%d %H:%M:%S")
consumo.loc[desde:hasta,:]

,Consumo máximo (kW),Consumo acumulado (kWh),Precio (€/kWh),Tarifa
2021-09-08 16:00:00,2.1,0.1,0.181,LLANO
2021-09-08 17:00:00,2.6,0.1,0.202,LLANO
2021-09-08 18:00:00,5.6,1.2,0.302,PUNTA


In [7]:
consumo.iloc[2:5,:]

,Consumo máximo (kW),Consumo acumulado (kWh),Precio (€/kWh),Tarifa
2021-09-08 16:00:00,2.1,0.1,0.181,LLANO
2021-09-08 17:00:00,2.6,0.1,0.202,LLANO
2021-09-08 18:00:00,5.6,1.2,0.302,PUNTA


Es importante recordar que cuando pasamos un intervalo a <code>iloc</code>, excluye el valor más alto, mientras que <code>loc</code> no hace esto.

Además, podemos combinar ambos tipos de selección:

In [8]:
consumo.loc[desde:hasta,columnas]

,Consumo acumulado (kWh),Precio (€/kWh)
2021-09-08 16:00:00,0.1,0.181
2021-09-08 17:00:00,0.1,0.202
2021-09-08 18:00:00,1.2,0.302


## Selección por filtrado de columna

Todo lo que hemos visto hasta ahora es tan sólo un repaso de lo que vimos en su momento al introducir Pandas. La verdadera potencia de la propiedad <code>.loc[]</code> está en la posibilidad de filtrar filas que cumplan una determinada condición en una de sus columnas. Por ejemplo, si queremos filtrar aquellas filas que correspondan a la tarifa punta, lo podemos hacer con <code>.loc[condicion,columnas]</code> de la siguiente manera:

In [9]:
consumo.loc[ consumo["Tarifa"] == "PUNTA", : ] 

,Consumo máximo (kW),Consumo acumulado (kWh),Precio (€/kWh),Tarifa
2021-09-08 18:00:00,5.6,1.2,0.302,PUNTA
2021-09-08 19:00:00,1.4,0.2,0.314,PUNTA


O quizás queremos ver aquellos intervalos en los que el consumo punta supera un determinado valor:

In [10]:
consumo.loc[ consumo["Consumo máximo (kW)"] > 3, : ] 

,Consumo máximo (kW),Consumo acumulado (kWh),Precio (€/kWh),Tarifa
2021-09-08 14:00:00,3.2,1.5,0.179,LLANO
2021-09-08 18:00:00,5.6,1.2,0.302,PUNTA


Las posibilidades son infinitas. Podemos incluso combinar condiciones con los operadores <code>&</code> (Y) y <code>|</code> (O). Para ver aquellos intervalos en hora punta <b>y</b> con un consumo máximo mayor que 3 kW haremos:

In [11]:
consumo.loc[ (consumo["Consumo máximo (kW)"] > 3) & ( consumo["Tarifa"] == "PUNTA"), : ] 

,Consumo máximo (kW),Consumo acumulado (kWh),Precio (€/kWh),Tarifa
2021-09-08 18:00:00,5.6,1.2,0.302,PUNTA


Para ver aquellos intervalos en hora punta <b>o</b> con un consumo máximo mayor que 3 kW haremos:

In [12]:
consumo.loc[ (consumo["Consumo máximo (kW)"] > 3) | ( consumo["Tarifa"] == "PUNTA"), : ] 

,Consumo máximo (kW),Consumo acumulado (kWh),Precio (€/kWh),Tarifa
2021-09-08 14:00:00,3.2,1.5,0.179,LLANO
2021-09-08 18:00:00,5.6,1.2,0.302,PUNTA
2021-09-08 19:00:00,1.4,0.2,0.314,PUNTA


Aquí podríamos estar tentados de usar los operadores <code>and</code> y <code>or</code> de Python, pero por la forma en la que está implementado Pandas, no se puede hacer esto.

<div style="background-color:lightpink; padding:1em"><b>Ejercicio 1</b><br/>
    <p>Represente una gráfica en la que se observe el consumo máximo y el consumo acumulado como una gráfica de lineas, sólo en horas de tarifa llana.</p>
    <ul>
    </ul>
</div>

## Conclusiones

En esta libreta hemos visto las operaciones básicas de filtrado. Si bien es un número muy pequeño de operaciones, éstas ofrecen una gran potencia a la hora de manipular datos. Son una parte esencial del análisis de datos, y están presentes en otras tecnologías también. La operación <code>SELECT</code> es, por ejemplo, la principal funcionalidad del lenguaje SQL, utilizado en bases de datos. Pandas simplifica y hace intuitiva esta operación. De hecho, estas operaciones (junto a la posibilidad de aplicar funciones sobre las filas, como veremos más adelante) son la clave del triunfo de Pandas en analítica de datos.